In [6]:
import torch
import torchvision.models as models
import torch.nn.functional as F
from torch import nn
import math
import os

In [7]:
import random
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np

In [8]:
print(torch.__version__)

2.0.1+cu117


In [9]:
torch.cuda.is_available()

True

In [10]:
#to get the current working directory
directory = os.getcwd()

print(directory)

/home/aandre/fmri_project


In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [12]:
from models import decoder2

model = decoder2.Decoder2()
model.cuda()
print(model)


Decoder2(
  (fc_output): Linear(in_features=13459, out_features=25088, bias=True)
  (conv1): Conv2d(128, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (upsample1): UpsamplingNearest2d(scale_factor=2.0, mode='nearest')
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 48, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (upsample2): UpsamplingNearest2d(scale_factor=2.0, mode='nearest')
  (bn2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(48, 48, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu3): ReLU()
  (upsample3): UpsamplingNearest2d(scale_factor=2.0, mode='nearest')
  (bn3): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(48, 3, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (sigmoid): Sigmoid()
  (upsample4): UpsamplingNearest2d(scale_factor=2.0, mode='

In [13]:
test_input = torch.rand(13459).cuda()
output_test = model(test_input)

In [14]:
output_test.shape

torch.Size([1, 3, 112, 112])

>Dataloader decoder

In [21]:
import cv2
import numpy as np

class MyDataset(Dataset):
    def __init__(self, image_dir, label_dir, num_images_per_sample=32):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.sample_ids = os.listdir(self.label_dir)
        random.shuffle(self.sample_ids)
        
    def __len__(self):
        return len(self.sample_ids)
    
    def __getitem__(self, idx):
        subfolder_id = self.sample_ids[idx]
        subfolder_id = subfolder_id.replace("label", "").split(".")[0]
    
        image_name = 'frame_{:03d}_{:03d}.jpg'.format(int(subfolder_id), 16)
        image_path = os.path.join(self.image_dir, image_name)
        im = cv2.imread(image_path)
        im_resized = cv2.resize(im, (112, 112), interpolation=cv2.INTER_LINEAR)
        im_tensor = torch.from_numpy(im_resized)
        fmri_name = 'label{}.npy'.format(int(subfolder_id))
        fmri_path = os.path.join(self.label_dir, fmri_name)
        fmri = np.load(fmri_path)
        return fmri, im_tensor
        
        return image_batch, label
    
    
image_dir = '/media/miplab-nas2/Data3/andre/stimuli_final'
label_dirs = 'labels2/1'

train_dataset = MyDataset(image_dir, label_dirs)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

print(len(train_dataset))

3599


In [19]:
from torchvision.models import vgg16
from models.p3d_model import *
import torchvision.transforms as transforms



# Example data
x = torch.randn(1, 3, 112, 112).cuda()  # Ground truth frame
x_hat = torch.randn(1, 3, 112, 112).cuda().requires_grad_(True)  # Reconstructed frame

# Example VGG model
vgg_model = vgg16(pretrained=True)  # Select a subset of VGG layers
vgg_model = vgg_model.cuda()
vgg_model.eval()


# Weight coefficients
beta = 0.5
gamma = 0.3
delta = 0.2

# Perceptual Similarity Loss (Lim)
def perceptual_similarity_loss(x, x_hat, vgg_model):
    x = x.view(1, 3, 112, 112)
    x = x.cuda().float()  # Convert input tensor to float
    x_hat = x_hat.view(1, 3, 112, 112)
    x_hat = x_hat.cuda().float()  # Convert reconstructed tensor to float
    
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    # Normalize the tensors
    x = normalize(x)
    x_hat = normalize(x_hat)
    
    x_features = vgg_model(x)
    x_hat_features = vgg_model(x_hat)
    lim_loss = nn.MSELoss()(x_features, x_hat_features)
    return lim_loss
# Regularization Term (LR)

def total_variation_loss(x_hat):
    diff_h = torch.abs(x_hat[:, :, :, 1:] - x_hat[:, :, :, :-1])
    diff_v = torch.abs(x_hat[:, :, 1:, :] - x_hat[:, :, :-1, :])
    tv_loss = torch.mean(diff_h) + torch.mean(diff_v)
    tv_loss.retain_grad()
    return tv_loss

def spatial_group_lasso_regularization(model):
    lasso_loss = 0
    for name, param in model.named_parameters():
        if 'fc_output' in name:  # Assuming the first FC layer is named 'fc1'
            lasso_loss += torch.norm(param, p=2)
    return lasso_loss

# Supervised Decoder Loss
def supervised_decoder_loss(x, x_hat, vgg_model, beta, gamma, delta, model):
    lim_loss = perceptual_similarity_loss(x, x_hat, vgg_model)
    tv_loss = total_variation_loss(x_hat)
    lasso_loss = spatial_group_lasso_regularization(model)
    loss = beta * lim_loss + delta * (tv_loss + lasso_loss)
    return loss

supervised_decoder_loss(x, x_hat, vgg_model, beta, gamma, delta, model)

lim tensor(2.5435, device='cuda:0', grad_fn=<MseLossBackward0>)
tv tensor(2.2516, device='cuda:0', grad_fn=<AddBackward0>)
lasso tensor(92.2395, device='cuda:0', grad_fn=<AddBackward0>)


tensor(20.1700, device='cuda:0', grad_fn=<AddBackward0>)

In [36]:
x_hat = torch.randn(1, 3, 112, 112).cuda().requires_grad_(True)

In [37]:
perceptual_similarity_loss(x, x_hat, vgg_model)

tensor(0.8036, device='cuda:0', grad_fn=<MseLossBackward0>)

In [38]:
total_variation_loss(x_hat)

tensor(2.2532, device='cuda:0', grad_fn=<AddBackward0>)

In [39]:
spatial_group_lasso_regularization(model)

tensor(92.2348, device='cuda:0', grad_fn=<AddBackward0>)

In [40]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [41]:
import torch.optim as optim
import math

initial_lr = 1e-3
lr_factor = 5
lr_step_size = 25

def get_learning_rate(epoch):
    return initial_lr / math.pow(lr_factor, epoch // lr_step_size)

optimizer = optim.Adam(model.parameters(), lr=get_learning_rate(0))
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=get_learning_rate)


In [ ]:
num_epochs = 100
print_interval = 10
losses_tot = []
min_loss = 10000000000000 #max

#scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.85)

for epoch in range(num_epochs):
    for i, (fmri, image) in enumerate(train_dataloader):
        optimizer.zero_grad()
        images = image.to(device).float()
        fmri = fmri.to(device).float()
        x_hat = model(fmri)
        loss = supervised_decoder_loss(image, x_hat, vgg_model, beta, gamma, delta)
        losses_tot.append(loss.mean().item())
        if (loss.mean().item()) < min_loss:
            min_loss = loss.mean().item()
            print('saving model with loss =', min_loss)
            torch.save(model.state_dict(), 'saved_model/decoder2.pth')
        loss.mean().backward()
        optimizer.step()

        # Print training statistics
        if i % print_interval == 0:
            print(f"Epoch [{epoch}/{num_epochs}], Step [{i}/{len(train_dataloader)}], Loss: {loss.mean().item():.4f}")

    # Update learning rate at the end of each epoch
    scheduler.step()